## PUC Minas
### Pós-Graduação em Ciência de Dados e Big Data
#### Avaliação Final - Modelagem e Preparação de Dados para Aprendizado de Máquina

**Aluno(s):**

In [2]:
# Luciano Augusto Scherer Contri

### Base de Dados

**Descriçao de Atributos**

- age: idade
- workclass: classe de trabalho
- education: nível educacional
- education-num: anos de educação
- marital-status: estado civil
- occupation: profissão
- race: etnia
- sex: gênero
- capital-gain: ganho de capital
- capital-loss: perda de capital
- hours-per-week: horas de trabalho por semana
- native-country: país de origem

**Contexto dos Dados**

O dataset apresenta dados de um problema de classificação onde o objetivo é prever se a pessoa da observação ganha mais de 50k dólares por ano ou não.

## Atividades

**Dados**

In [3]:
# Carregue o dataset fornecido ('adult_final.csv')

In [4]:
import pandas as pd

adult_final = pd.read_csv('../../Datasets/adult_final.csv', sep=',')
adult_final

**1. Apresente o tipo das variáveis.**

In [5]:
adult_final.dtypes

**2.Apresente de forma gráfica e numérica a análise exploratória das variáveis _education_ e _race_.**

In [6]:
adult_final['education'].value_counts().plot(kind='bar')

In [7]:
# porcentagem 
adult_final['education'].value_counts(normalize=True) * 100

In [8]:
adult_final['race'].value_counts().plot(kind='bar')

In [9]:
# porcentagem
adult_final['race'].value_counts(normalize=True) * 100

**3. Apresente as métricas estatísticas (média, moda, etc.) e histograma das variáveis _age_ e _hours-per-week_.**

In [10]:
# agregações
adult_final.agg({'age': ['mean', 'median', 'std', 'var', 'min', 'max'],
                 'hours-per-week': ['mean', 'median', 'std', 'var', 'min', 'max']})

**4. Apresente 2 análises multivaridas entre variáveis a sua escolha.**

In [11]:
# média de idade por classe de trabalho
adult_final.groupby('workclass')[['capital-gain', 'age']].mean().sort_values(by='age')

In [12]:
adult_final.groupby('occupation')['capital-gain'].mean().sort_values()

In [13]:
# workclass por média de idade
adult_final.groupby('workclass')['age'].mean().sort_values()

In [14]:
adult_final.groupby('native-country')['education-num'].mean().sort_values()

**5. Apresente a soma de _NaN_ de cada coluna da base de dados.**

In [15]:
adult_final.isna().sum()

**6. Trate os _NaN_ de todas as colunas como achar conveniente (explique). Em seguida, mostre que nenhuma coluna apresenta _NaN_ ao final do processo.**

##### tratamento de NaN workclass

In [16]:
# capturando o index de um null para verificar o resultado posterior
index_nan = adult_final[adult_final['workclass'].isna()].index

# observar um exemplo de linha com NaN
adult_final.loc[index_nan].head(5)

In [17]:
# substituindo os valores NaN para cada coluna

# Calcula a média de idade para cada categoria de workclass
mean_age_by_workclass = adult_final.groupby('workclass')['age'].mean()


# Função para encontrar a workclass com a média de idade mais próxima
def find_closest_workclass_age(mean_age_by_workclass, age):
    closest_workclass = None
    min_distance = float('inf')
    for workclass, mean_age in mean_age_by_workclass.items():
        distance = abs(mean_age - age)
        if distance < min_distance:
            min_distance = distance
            closest_workclass = workclass
    return closest_workclass


# Preenche os valores nulos na coluna 'workclass' com a workclass correspondente à média de idade mais próxima
for index, row in adult_final.iterrows():
    if pd.isnull(row['workclass']):
        closest_workclass = find_closest_workclass_age(mean_age_by_workclass, row['age'])
        adult_final.at[index, 'workclass'] = closest_workclass

# observaros antigos NaN
adult_final.loc[index_nan].head(5)

Todos os valores NaN foram substituídos pela classe de trabalho com a média de idade mais próxima.

##### tratamento de NaN occupation

In [18]:
# capturando o index de um null para verificar o resultado posterior
index_nan = adult_final[adult_final['occupation'].isna()].index

# observar um exemplo de linha com NaN
adult_final.loc[index_nan].head(5)

In [19]:
# Calcula a média de idade para cada categoria de workclass
mean_capital_gain_by_occupation = adult_final.groupby('occupation')['capital-gain'].mean()


def find_closest_occupation_capital_gain(mean_capital_gain_by_occupation, capital_gain):
    closest_occupation = None
    min_distance = float('inf')
    for occupation, mean_capital_gain in mean_capital_gain_by_occupation.items():
        distance = abs(mean_capital_gain - capital_gain)
        if distance < min_distance:
            min_distance = distance
            closest_occupation = occupation
    return closest_occupation


# Preenche os valores nulos na coluna 'occupation' com a occupation correspondente ao ganho de capital mais próximo
for index, row in adult_final.iterrows():
    if pd.isnull(row['occupation']):
        closest_occupation = find_closest_occupation_capital_gain(mean_capital_gain_by_occupation, row['capital-gain'])
        adult_final.at[index, 'occupation'] = closest_occupation


In [20]:
# observar os antigos NaN
adult_final.loc[index_nan].head(5)

Todos os valores NaN foram substituídos pela ocupação com o ganho de capital médio por ocupação mais próximo.

In [21]:
adult_final.isna().sum()

##### tratamento de NaN native-country

Eu não vejo muita utilidade em preencher os valores NaN da coluna 'native-country' com base em outras colunas, pois o numero de valores NaN é muito pequeno em relação ao total de registros. Portanto prefiro remover esses registros.

In [22]:
adult_final.dropna(inplace=True)
adult_final.isna().sum()

In [23]:
adult_final.shape

**7. Aplique _Ordinal Encoding_ em uma variável categórica ordinal.**

In [24]:
# ordinal encoding na coluna 'education'
from sklearn.preprocessing import OrdinalEncoder

adult_final['education'] = OrdinalEncoder().fit_transform(adult_final[['education']])
adult_final['education']

**8. Aplique _One Hot Encoding_ em uma variável categórica nominal.**

In [25]:
# categorias nominais: workclass, occupation e native-country

# one hot encoding na coluna 'workclass'
adult_final = pd.get_dummies(adult_final, columns=['workclass'])

In [26]:
adult_final

**9. Aplique uma técnica de _oversampling_ (classe minoritária) e uma de _undersampling_ (classe majoritária). Apresente a mudança de volumetria (antes e depois). Se necessário, lembre-se de tratar as variáveis categóricas de forma adequada caso deseje usar um método mais robusto (SMOTE, por exemplo). Se for o caso, utilize PCA para visualizar os dados de forma bidimensional (antes e depois da amostragem).**

In [27]:
# tratamento de variáveis categóricas

# one hot encoding nas colunas restantes menos a coluna target
cat_columns = adult_final.select_dtypes(include=['object']).columns
cat_columns = cat_columns.drop('target')
cat_columns

In [28]:
adult_final = pd.get_dummies(adult_final, columns=cat_columns)
# coluna target para numérico
adult_final['target'] = adult_final['target'].map({'<=50K': 0, '>50K': 1})

In [29]:
adult_final.dtypes.value_counts()

Gerar um Dataset balanceado com oversampling e outro com undersampling

In [30]:
adult_final['target'].value_counts()

##### Oversampling com SMOTE

Smote é uma técnica de oversampling que cria novos registros sintéticos da classe minoritária. Temos 24283 registros da classe <=50K e 7695 registros da classe >50K. Portanto, o smote gerará 16588 registros sintéticos da classe >50K para igualar o número de registros da classe <=50K.

In [31]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X = adult_final.drop('target', axis=1)
y = adult_final['target']

X_smote, y_smote = smote.fit_resample(X, y)

In [32]:
# juntando os resultados do SMOTE
adult_final_smote = pd.concat([X_smote, y_smote], axis=1)
adult_final_smote['target'].value_counts()

##### Undersampling com RandomUnderSampler

RandomUnderSampler é uma técnica de undersampling que remove registros aleatórios da classe majoritária. Temos 24283 registros da classe <=50K e 7695 registros da classe >50K. Portanto, o RandomUnderSampler removerá 16588 registros da classe <=50K para igualar o número de registros da classe >50K.

In [33]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(sampling_strategy='auto')
X_rus, y_rus = rus.fit_resample(X, y)

In [34]:
# juntando os resultados do RandomUnderSampler
adult_final_rus = pd.concat([X_rus, y_rus], axis=1)
adult_final_rus['target'].value_counts()

##### PCA para visualização dos dados

In [43]:
from sklearn.preprocessing import StandardScaler
# PCA para visualização dos dados

from sklearn.decomposition import PCA

pca = PCA(n_components=2)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_smote = scaler.fit_transform(X_smote)
x_scaled_rus = scaler.fit_transform(X_rus)

X_pca = pca.fit_transform(X_scaled)
X_smote_pca = pca.fit_transform(X_scaled_smote)
X_rus_pca = pca.fit_transform(x_scaled_rus)

In [44]:
X_pca  

In [45]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Criando DataFrames
df_no_sampling = pd.DataFrame(X_pca, columns=['Componente 1', 'Componente 2'])
df_no_sampling['y'] = y

df_smote = pd.DataFrame(X_smote_pca, columns=['Componente 1', 'Componente 2'])
df_smote['y'] = y_smote

df_rus = pd.DataFrame(X_rus_pca, columns=['Componente 1', 'Componente 2'])
df_rus['y'] = y_rus

# Lista de DataFrames para plotagem
datasets = [
    (df_no_sampling, 'No Sampling'),
    (df_smote, 'SMOTE'),
    (df_rus, 'RUS')
]

# Cores para a legenda
colors = ['red', 'blue']  # Modifique conforme necessário para corresponder aos seus dados
labels = ['<=50K', '>50K']  # Atualize conforme a necessidade

# Plotando cada um dos conjuntos de dados
for df, name in datasets:
    fig, ax = plt.subplots()
    scatter = ax.scatter(df['Componente 1'], df['Componente 2'], c=df['y'], cmap='coolwarm', alpha=0.6)

    # Criar legendas manualmente
    handles = [mpatches.Patch(color=color, label=label) for color, label in zip(colors, labels)]

    # Aplicar o mapeamento aos rótulos
    ax.legend(handles=handles, loc="upper right", title="Classes")

    # Configurar os eixos
    ax.set_xlabel('Componente 1')
    ax.set_ylabel('Componente 2')
    ax.set_title(name)

    # Mostrar o gráfico
    plt.show()

**10. Aplique _One Hot Encoding_ nas variáveis _race_ e _sex_. Junte ao resultado _TODAS_ as outras variáveis númericas (_age_, _education-num_, _capital-gain_, _capital-loss_ e _hours-per-week_). Utilize o dataset resultante no algoritmo t-SNE e reduza a dimensionalidade à 2 componentes (padrão do algoritmo). Plote o resultado diferenciando os pontos pela classe (atributo _target_).**

 One Hot Encoding nas variáveis já foram feitos anteriormente para a execução do smote e rus. 

##### t-SNE

In [59]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

tsne = TSNE(n_components=2)

In [60]:
data = adult_final[['age', 'education-num', 'capital-gain', 'capital-loss',
                    'hours-per-week', 'race_Amer-Indian-Eskimo',
                    'race_Asian-Pac-Islander', 'race_Black',
                    'race_Other', 'race_White', 'sex_Female', 'sex_Male']]
X_tsne = tsne.fit_transform(data)

In [61]:
data_smote = adult_final_smote[['age', 'education-num', 'capital-gain', 'capital-loss',
                                'hours-per-week', 'race_Amer-Indian-Eskimo',
                                'race_Asian-Pac-Islander', 'race_Black',
                                'race_Other', 'race_White', 'sex_Female', 'sex_Male']]
X_tsne_smote = tsne.fit_transform(data_smote)

In [62]:
data_rus = adult_final_rus[['age', 'education-num', 'capital-gain', 'capital-loss',
                            'hours-per-week', 'race_Amer-Indian-Eskimo',
                            'race_Asian-Pac-Islander', 'race_Black',
                            'race_Other', 'race_White', 'sex_Female', 'sex_Male']]
X_tsne_rus = tsne.fit_transform(data_rus)

In [76]:
target = adult_final['target']
target_smote = adult_final_smote['target']
target_rus = adult_final_rus['target']

datasets = [(X_tsne, target, 'No Sampling'), (X_tsne_smote, target_smote, 'Smote'), 
            (X_tsne_rus, target_rus, 'Rus')]

for X, target, name in datasets:
    fig, ax = plt.subplots()

    scatter = ax.scatter(X[:, 0], X[:, 1], c=target, cmap='coolwarm')

    # Create a mapping dictionary
    labels = ['<=50K', '>50K']

    # Get the legend elements
    handles, _ = scatter.legend_elements()

    # Apply the mapping to the labels
    ax.legend(handles, labels, loc="upper right", title="Classes")

    # Set the axes
    plt.xlabel('Componente 1')
    plt.ylabel('Componente 2')
    plt.title(name)

    # Show the plot
    plt.show()